In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def polinomial_func(x_step, x, y, deg=2):
    p = np.polyfit(x, y, deg)
    z = np.poly1d(p)
    curve = z(x_step)
    return curve

def mover_elemento_al_final(arr, indice):
    if 0 <= indice < len(arr):
        elemento = arr.pop(indice)
        arr.append(elemento)
        return arr
    else:
        print("Índice fuera de rango")

In [ ]:
workload = 'h264_enc'
df = pd.read_csv('metrics_{}.csv'.format(workload))
df.head()

In [ ]:
def generar_tendencias():    
    l3_cache = ['1MB', '2MB', '4MB', '8MB']
    l2_cache = ['128kB', '256KB', '512KB', '1MB']
    decode_width = ['1', '2', '4', '8']
    num_alu = ['2', '4', '8', '16']
    label_y = ['CPI (Cycles Per Instruction)', 'Energy [J]', 'EDP (Energy-Delay Product)']
    metric = ['CPI', 'Energy', 'EDP']

    for j in range(3):
        if j == 0:
            color = 'b'
        elif j == 1:
            color = 'g'
        elif j == 2:
            color = 'r'
        for i in range(4):
            data = []
            if i == 0:
                l1 = "l2_cache=128kB_decode_width=1_num_fu_intALU=16"
                l2 = ""
                eje_x = l3_cache
                label_x = 'Tamaño de la cache L3'
            elif i == 1:
                l1 = "decode_width=1_num_fu_intALU=16"
                l2 = "l3_cache=1MB"
                eje_x = l2_cache
                label_x = 'Tamaño de la cache L2'
            elif i == 2:
                l1 = "num_fu_intALU=16"
                l2 = "l3_cache=1MB_l2_cache=128kB"
                eje_x = decode_width
                label_x = 'Ancho de decodificación de CPU'
            elif i == 3:
                l1 = "l3_cache=1MB_l2_cache=128kB_decode_width=1"
                l2 = ""
                eje_x = num_alu
                label_x = 'Número de ALUs'

            for it, item in enumerate(df[metric[j]]):
                if l1 in df['label'][it] and l2 in df['label'][it]:
                    print(df['label'][it])
                    data.append(item)

            if i == 1:
                data = mover_elemento_al_final(data, 1)
            elif i == 3:
                data = mover_elemento_al_final(data, 0)

            x_step = np.linspace(0, len(data)-1, 100)
            plt.figure()
            plt.scatter(eje_x, data, s=60, marker='s', c=color, edgecolors='k', zorder=3, clip_on=False)
            plt.plot(x_step, polinomial_func(x_step, np.arange(len(data)), data), c=color, linestyle='--')
            plt.xlabel(label_x, fontsize=14)
            plt.ylabel(label_y[j], fontsize=14)
            plt.xlim(0, len(data)-1)
            plt.savefig('{}_{}_{}.svg'.format(workload, metric[j], i), format='svg', bbox_inches='tight', pad_inches=0.1)
            plt.show()

In [ ]:
def generar_energia_cpi():    
    plt.figure()
    plt.scatter(df['Energy'], df['CPI'], s=60, c='y', edgecolors='k', zorder=3, clip_on=False)
    plt.xlabel('Energy [J]', fontsize=14)
    plt.ylabel('CPI (Cycles Per Instruction)', fontsize=14)
    plt.savefig('{}_energy_cpi.svg'.format(workload), format='svg', bbox_inches='tight', pad_inches=0.1)
    plt.show()